In [16]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns
import math

In [17]:
os.chdir('D:\\Moji fajlovi\\Master\\prog\\Project 5 - Agriculture Data Analysis')

In [30]:
data = pd.read_csv('database_biosense.csv',header=0, delimiter=';',skiprows=1)
data.head(25)

,Parcel,Area,Year,Crop,Seed production,Variety,Previous crop,Seed units,Manure,Fertiliser amount,...,Irrigation,Harvest,Seed,Fertiliser,Pesticide,Yield,Price,Revenue,Planting date,Harvest date
0,G-1/1,"63,38",2018,Maize,0,NS 205,Maize,184,0,"237,719575",...,9000,"11853,74",9450,"5037,277795","15202,25308","7,811454718","14,4","112484,9479",15.6.2018,15.10.2018
1,G-1/2,"23,01",2018,Barley,0,Salamandre,Maize,46,0,"238,6363636",...,0,"11762,74","8594,30605","7079,225979",1435,"7,146352313","15,8","112912,3665",28.9.2017,5.6.2018
2,G-2/1,"94,88",2018,Soybean,0,Gorstak,Maize,"100,33",0,"214,6331148",...,9000,"11549,5","10191,21689","7146,257137","11843,41404","3,388877141","37,35","126574,5612",19.4.2018,13.9.2018
3,G-2/2,"20,59",2018,Soybean,0,Wendy,Maize,"93,15",0,"140,8450704",...,0,"11571,08","5428,848956","4776,056338","12594,37882","3,71296746","37,35","138679,3346",17.4.2018,14.9.2018
4,G-3/1,"63,62",2018,Maize,0,DKC 5830,Wheat,60,"1,886199308","474,6934926",...,9000,"12801,04","19899,77051","19745,51399","4947,696479","14,60232631","14,4","210273,4989",17.4.2018,14.9.2018
5,G-3/2,"17,8",2018,Maize,0,DKC 5830,Wheat,14,"2,134831461","264,0449438",...,0,"12801,04","16595,81236","8427,921348","6112,939326","13,63539326","14,4","196349,6629",17.4.2018,14.9.2018
6,G-4/1,"63,32",2018,Soybean,0,Galeb,Maize,"74,22615287",0,"307,6437145",...,9000,"11571,08","7447,109918","10072,50158","11048,21854","3,975837018","37,35","148497,5126",13.4.2018,16.9.2018
7,G-4/2,"24,71",2018,Soybean,0,Galeb,Maize,"80,93889114",0,"273,7353298",...,0,"11571,08","7447,109918","10072,50158","11048,21854","4,068798057","37,35","151969,6074",13.4.2018,16.9.2018
8,G-5/1,"76,88",2018,Maize,0,DKC 5830,Soybean,"0,949531738",0,"386,4854318",...,9000,"12801,04","20071,65544","7823,860562","9002,979318","16,76378772","14,4","241398,5432",13.4.2018,16.9.2018
9,G-5/2,"20,98",2018,Maize,0,DKC 5830,Soybean,"0,81029552",0,"266,920877",...,0,"12801,04","19877,95148","8530,79123","7781,716397","11,23164919","14,4","161735,7483",13.4.2018,16.9.2018


In [19]:
def comma_to_dot(x):
    y = x.split(',')
    return float('.'.join(y))

In [11]:
type(data.loc[1,'Area'])

str

In [31]:
data['Area'] = data['Area'].apply(comma_to_dot)
data

,Parcel,Area,Year,Crop,Seed production,Variety,Previous crop,Seed units,Manure,Fertiliser amount,...,Irrigation,Harvest,Seed,Fertiliser,Pesticide,Yield,Price,Revenue,Planting date,Harvest date
0,G-1/1,63.3800,2018,Maize,0,NS 205,Maize,184,0,"237,719575",...,9000,"11853,74",9450,"5037,277795","15202,25308","7,811454718","14,4","112484,9479",15.6.2018,15.10.2018
1,G-1/2,23.0100,2018,Barley,0,Salamandre,Maize,46,0,"238,6363636",...,0,"11762,74","8594,30605","7079,225979",1435,"7,146352313","15,8","112912,3665",28.9.2017,5.6.2018
2,G-2/1,94.8800,2018,Soybean,0,Gorstak,Maize,"100,33",0,"214,6331148",...,9000,"11549,5","10191,21689","7146,257137","11843,41404","3,388877141","37,35","126574,5612",19.4.2018,13.9.2018
3,G-2/2,20.5900,2018,Soybean,0,Wendy,Maize,"93,15",0,"140,8450704",...,0,"11571,08","5428,848956","4776,056338","12594,37882","3,71296746","37,35","138679,3346",17.4.2018,14.9.2018
4,G-3/1,63.6200,2018,Maize,0,DKC 5830,Wheat,60,"1,886199308","474,6934926",...,9000,"12801,04","19899,77051","19745,51399","4947,696479","14,60232631","14,4","210273,4989",17.4.2018,14.9.2018
5,G-3/2,17.8000,2018,Maize,0,DKC 5830,Wheat,14,"2,134831461","264,0449438",...,0,"12801,04","16595,81236","8427,921348","6112,939326","13,63539326","14,4","196349,6629",17.4.2018,14.9.2018
6,G-4/1,63.3200,2018,Soybean,0,Galeb,Maize,"74,22615287",0,"307,6437145",...,9000,"11571,08","7447,109918","10072,50158","11048,21854","3,975837018","37,35","148497,5126",13.4.2018,16.9.2018
7,G-4/2,24.7100,2018,Soybean,0,Galeb,Maize,"80,93889114",0,"273,7353298",...,0,"11571,08","7447,109918","10072,50158","11048,21854","4,068798057","37,35","151969,6074",13.4.2018,16.9.2018
8,G-5/1,76.8800,2018,Maize,0,DKC 5830,Soybean,"0,949531738",0,"386,4854318",...,9000,"12801,04","20071,65544","7823,860562","9002,979318","16,76378772","14,4","241398,5432",13.4.2018,16.9.2018
9,G-5/2,20.9800,2018,Maize,0,DKC 5830,Soybean,"0,81029552",0,"266,920877",...,0,"12801,04","19877,95148","8530,79123","7781,716397","11,23164919","14,4","161735,7483",13.4.2018,16.9.2018


In [32]:
l = list(data)[7:23]
for i in l:
    data[i] = data[i].apply(comma_to_dot)
data

AttributeError: 'float' object has no attribute 'split'

In [29]:
type(data.loc[1,'Manure'])

str